In [ ]:
#from google.colab import drive

#drive.mount('/content/gdrive')

In [1]:
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np

PATH = Path('../data')
#PATH = Path('/content/gdrive/My Drive/ml-practico/data')

# Load the data

Con lo que aprendimos del notebook anterior

In [8]:
import pandas as pd

title_basics = pd.read_csv(PATH / 'title.basics.tsv', sep='\t')

movie_gross = pd.read_csv(PATH / 'movie_gross.csv')
# Calculamos el id de pelicula
movie_gross['tconst'] = movie_gross.movie_imdb_link.apply(lambda x: x.split('/')[4])
# Deduplicamos los registros
movie_gross = movie_gross.groupby('tconst').gross.max().reset_index()

title_ratings = pd.read_csv(PATH / 'title.ratings.tsv', sep='\t')

/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [9]:
title_basics.runtimeMinutes

0           1
1           5
2           4
3          12
4           1
           ..
7156875    \N
7156876    \N
7156877    \N
7156878    27
7156879    10
Name: runtimeMinutes, Length: 7156880, dtype: object

In [4]:
from itertools import chain
from collections import Counter

def parse_genres(genres):
    if isinstance(genres, float) or genres == r'\N': return ['no-genre']
    else: return genres.split(',')
    

# Convertimos runtimeMinutes a float. No se puede tener una columna de tipo int con NaN
title_basics.runtimeMinutes = (
    title_basics.runtimeMinutes.apply(lambda x: np.nan if not x.isdigit() else x).astype(float)
)

title_basics['genres'] = title_basics.genres.apply(parse_genres)

AttributeError: 'float' object has no attribute 'isdigit'

In [ ]:
title_basics = title_basics[
    # Dejamos tvSpecial, video y tvMovie por ahora, vamos a ver de que se tratan
    ~title_basics.titleType.isin(['tvEpisode', 'tvSeries', 'tvMiniSeries', 'videoGame', 'tvShort', 'short'])
    # Que tengan valor de runtimeMinutes
    & ~title_basics.runtimeMinutes.isna()
    # Menos de 3 horas y media para no descartar a titanic
    & (title_basics.runtimeMinutes <= 3.5 * 60)
    # Descartamos los shorts
    & title_basics.genres.apply(lambda x: 'Short' not in x)
    
]

In [ ]:
# Juntamos todo

movies = (
    title_basics.merge(movie_gross, on='tconst', how='left')
                .merge(title_ratings, on='tconst', how='left')
)

len(movies)

### Sanity check

Se ve igual que en el notebook anterior?

In [ ]:
movies.runtimeMinutes.hist(bins=40);

Miremos un caso individual (titanic)

In [ ]:
movies[movies.tconst == 'tt0120338']

# Hipótesis: el rating es buen predictor del gross

In [ ]:
len(movies)

In [ ]:
# Only gross and rating data frame
ogr_df = movies.dropna(subset=['averageRating', 'numVotes', 'gross']).dropna()

In [ ]:
ogr_df.plot.scatter('numVotes', 'averageRating', alpha=0.25)
plt.xscale('log')

In [ ]:
ogr_df['log(numVotes)'] = np.log10(ogr_df.numVotes)

In [ ]:
ogr_df.plot.scatter('averageRating', 'gross', c='log(numVotes)', cmap='Reds', alpha=0.25, figsize=(7,5))
plt.yscale('log');

No parece haber una correlacion directa, pero parece haber dos clusters diferenciados. Podrán explicarse con algun genero, u otra columna de los datos?

In [ ]:
(
    ogr_df[ogr_df.genres.apply(lambda x: 'Thriller'  in x)]
    .plot.scatter('averageRating', 'gross', c='log(numVotes)', cmap='Reds', alpha=0.25, figsize=(7,5))
)
plt.yscale('log');

Habrá mas como estos?

Habrá bias en la data que tiene gross?

In [ ]:
ogr_df.averageRating.plot.density(bw_method=0.5)
movies[~movies.averageRating.isna()].averageRating.plot.density(bw_method=0.5)

In [ ]:
plt.boxplot([ogr_df.averageRating, movies[~movies.averageRating.isna()].averageRating]);
plt.xticks([1, 2], ['ogr', 'all movies'])
plt.grid()

Si miramos numVotes, si hay una diferencia grande

In [ ]:
plt.boxplot([np.log10(ogr_df.numVotes), np.log10(movies[~movies.averageRating.isna()].numVotes)]);
plt.xticks([1, 2], ['ogr', 'all movies'])
plt.grid()

Pareciera que las colas de la distribucion no estan bien representadas

# Tiene sentido plantarlo como clasificacion: P(gross > X dollars)?

In [ ]:
np.log10(ogr_df.gross).plot.density(bw_method=0.05)

Parece haber un **sesgo grande** en la eleccion de las peliculas de movies_gross como para utilizarla para este proposito

Volvemos con el equipo de negocio y les proponemos predecir el rating como proxy. Se sienten comodes

# Si utilizamos la probabilidad de que reciba al menos un rating de X?

In [ ]:
movies.dropna(subset=['averageRating']).averageRating.plot.density(bw_method=0.05)

In [ ]:
# La distribucion es amplia, parece ser interesante
np.percentile(movies.averageRating.dropna(), [25, 50, 75, 85, 95])

In [ ]:
movies.head()

## 🤔 Para pensar 

Cuantos votos tiene que tener una pelicula para usarla en training / testing?

Qué metrica usamos? 
* Accuracy? 
* ROC AUC?
* f1?
* Otra?

## Train test set split

In [ ]:
movies['startYear'] = movies['startYear'].apply(lambda x: np.nan if x == r'\N' else int(x))

In [ ]:
movies.groupby('startYear').tconst.agg('count').plot()

In [ ]:
movies = movies[movies.startYear > 1970].dropna(subset=['averageRating'])

In [ ]:
train_df = movies[movies.startYear <= 2017]
test_df = movies[movies.startYear > 2017]

len(train_df), len(test_df), len(test_df) / len(train_df)

In [ ]:
y_train = (train_df.averageRating >= 7.4).values
y_test = (test_df.averageRating >= 7.4).values

In [ ]:
y_train.sum() / len(y_train), y_test.sum() / len(y_test)

# Feature engineering 101

## Hay relacion entre el genero y la pelicula?

In [ ]:
# Vamos a usar todo como listas de diccionarios para poder usar el ecosistema de sklearn de forma sencilla

train_docs = train_df.to_dict(orient='records')
test_docs = test_df.to_dict(orient='records')

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

class GenreDummies(BaseEstimator, TransformerMixin):
    def fit(self, X, y): return self

    def transform(self, X):
        res = []
        for e in X:
            res.append({g: 1 for g in e['genres']})
        return res        

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression

In [ ]:
from sklearn.metrics import roc_auc_score

def test_pipe(pipe):
    return {
        'train_auc': roc_auc_score(y_train, pipe.predict_proba(train_docs)[:, 1]),
        'test_auc': roc_auc_score(y_test, pipe.predict_proba(test_docs)[:, 1])
    }

In [ ]:
pipe = make_pipeline(
    GenreDummies(), DictVectorizer(sparse=False), StandardScaler(), LogisticRegression()
)

pipe.fit(train_docs, y_train);

In [ ]:
test_pipe(pipe)

## Cómo controlamos por el año?

Tiene sentido?

In [ ]:
from datetime import datetime
from sklearn.base import BaseEstimator, TransformerMixin

class YearsAgo(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.now = datetime.now().year
        
    def fit(self, X, y): return self

    def transform(self, X):
        res = []
        for e in X:
            res.append({'years_ago': self.now - int(e['startYear'])})
        return res

In [ ]:
from sklearn.pipeline import make_union

pipe = make_pipeline(
    make_union(
        make_pipeline(YearsAgo(), DictVectorizer(sparse=False)),
        make_pipeline(GenreDummies(), DictVectorizer(sparse=False))
    ),
    StandardScaler(),
    LogisticRegression()
)

pipe.fit(train_docs, y_train);

In [ ]:
test_pipe(pipe)

No afecta mucho la metrica.

**Challenge**: esto realmente controla algo? sirve para algo?

## Cómo podemos incorporar información de la crew?

### ETL

In [ ]:
crew_df = pd.read_csv(PATH / 'title.crew.tsv', sep='\t')
principals_df = pd.read_csv(PATH / 'title.principals.tsv', sep='\t')
names_df = pd.read_csv(PATH / 'title.basics.tsv', sep='\t')

In [ ]:
names_df.head()

In [ ]:
crew_df.head()

In [ ]:
principals_df.head()

In [ ]:
principals_df.category.value_counts()

En las peliculas hay mas de un director

El director puede estar en ordering random 

ejemplos tt4898864 (10 directores)

titanic tt0120338 director en ordering 5 

In [ ]:
# Me quedo solo con los que fueron directores
movies_directors = principals_df[principals_df.category == 'director'].copy()
# Calculo un ranking por pelicula segun el ordering
movies_directors['director_rank'] = (
    movies_directors.sort_values('ordering')
                    .groupby('tconst')
                    .cumcount()
)

In [ ]:
# Me quedo con el "director principal" por pelicula
movies_directors = movies_directors[movies_directors.director_rank == 0]

In [ ]:
# Sanity check, no debería haber repetidos
movies_directors.tconst.value_counts().max()

In [ ]:
# me quedo solo con la columna del director
movies_directors = (
    movies_directors.rename(columns={'nconst': 'director'})
    [['tconst', 'director']]
)

In [ ]:
movies_directors.head()

In [ ]:
train_df = train_df.merge(movies_directors, on='tconst', how='left').fillna('-')
test_df = test_df.merge(movies_directors, on='tconst', how='left').fillna('-')

In [ ]:
director_movies_distr = (
    train_df[train_df.director != '-']
            .director
            .value_counts()
            .value_counts()
            .sort_index()
)

director_movies_distr.plot(style='-o', logx=True, logy=True)

In [ ]:
(director_movies_distr.cumsum() / director_movies_distr.sum()).head(10)

In [ ]:
(train_df.director == '-').sum() / len(train_df)

El 9% de los casos no tenemos info del director.

La mayoria (64%) de los directores hicieron una peli nada mas

In [ ]:
# Sanity check: como se ve titanic?
train_df[train_df.tconst == 'tt0120338']

In [ ]:
(train_df.director.value_counts() >= 2).sum()

### Transformer & experiments

In [ ]:
from collections import Counter

class DirectorFeatures(BaseEstimator, TransformerMixin):
    def __init__(self, min_cnt_movies = 2):
        self.min_cnt_movies = min_cnt_movies
        
    def fit(self, X, y): 
        # Esto no es la forma mas elegante, pero es mas comodo y a esta altura priorizo la comodidad
        # Llevamos las cosas de nuevo a un DataFrame y calculamos features por director
        directors_stats = (
            pd.DataFrame(X)
              .groupby('director')
              .agg({
                  'tconst': 'count', 
                  'averageRating': ['mean', 'max', 'min'], 
                  'numVotes': ['mean', 'min', 'max']}
              )
        )
        
        # Para hacer flattening de las columnas
        # https://stackoverflow.com/questions/14507794/pandas-how-to-flatten-a-hierarchical-index-in-columns
        directors_stats.columns = [
            '_'.join(i) 
            for i in zip(directors_stats.columns.get_level_values(1), directors_stats.columns.get_level_values(0))
        ]
        
        # Guardamos las estadisticas
        self.directors_stats_ = directors_stats
        
        # Diccionario con los datos para los directores comunes
        self.directors_stats_lk_ = (
            directors_stats[directors_stats.count_tconst >= self.min_cnt_movies].to_dict(orient='index')
        )
        
        # Valor default para los que consideramos que tenemos demasiado poca data
        self.default_ = directors_stats[directors_stats.count_tconst < self.min_cnt_movies].mean(0).to_dict()
        if self.min_cnt_movies > 1:
            self.default_ = directors_stats[directors_stats.count_tconst < self.min_cnt_movies].mean(0).to_dict()
        else:
            self.default_ = directors_stats.mean(0).to_dict()
        return self

    def transform(self, X):
        res = []
        for e in X:
            if e['director'] in self.directors_stats_lk_:
                res.append(self.directors_stats_lk_[e['director']])
            else:
                res.append(self.default_)
        return res            

In [ ]:
train_docs = train_df.to_dict(orient='records')
test_docs = test_df.to_dict(orient='records')

In [ ]:
pipe = make_pipeline(
    DirectorFeatures(), DictVectorizer(sparse=False), StandardScaler(), LogisticRegression()
)

In [ ]:
pipe.fit(train_docs, y_train);

In [ ]:
test_pipe(pipe)

In [ ]:
from sklearn.pipeline import make_union

pipe = make_pipeline(
    make_union(
        make_pipeline(YearsAgo(), DictVectorizer(sparse=False)),
        make_pipeline(GenreDummies(), DictVectorizer(sparse=False)),
        make_pipeline(DirectorFeatures(min_cnt_movies=4), DictVectorizer(sparse=False))
    ),
    StandardScaler(),
    LogisticRegression()
)

In [ ]:
pipe.fit(train_docs, y_train);

In [ ]:
test_pipe(pipe)

# Para seguir

* Como podes explotar aún mas el cast and crew?
* Como podes manejar el caso donde no sabemos mucho sobre el director? Esta bien min_count=2? debería ser mayor? menor?
* Qué otras familias de modelos no-lineales podemos probar?